In [1]:
# data analysis
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

- Step 1.採集數據

In [2]:
# 讀取 csv 檔案
train_df = pd.read_csv('data/titanic/train.csv')
test_df = pd.read_csv('data/titanic/test.csv')

# 合併資料
combine = [train_df, test_df]

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [18]:
train_df['Family_size'] = train_df['SibSp'] + train_df['Parch']+1

deplicate_ticket = []
for tk in train_df.Ticket.unique():
    tem = train_df.loc[train_df.Ticket == tk, 'Fare']
    if tem.count() > 1:
        deplicate_ticket.append(train_df.loc[train_df.Ticket == tk, ['Name','Ticket', 'Age', 'Fare', 'Family_size', 'Pclass','Survived']])
deplicate_ticket = pd.concat(deplicate_ticket)    
deplicate_ticket.tail(50)

,Name,Ticket,Age,Fare,Family_size,Pclass,Survived
453,"Goldenberg, Mr. Samuel L",17453,49.0,89.1042,2,1,1
849,"Goldenberg, Mrs. Samuel L (Edwiga Grabowska)",17453,NaN,89.1042,2,1,1
476,"Renouf, Mr. Peter Henry",31027,34.0,21.0000,2,2,0
726,"Renouf, Mrs. Peter Henry (Lillian Jefferys)",31027,30.0,21.0000,4,2,1
496,"Eustis, Miss. Elizabeth Mussey",36947,54.0,78.2667,2,1,1
591,"Stephenson, Mrs. Walter Bertram (Martha Eustis)",36947,52.0,78.2667,2,1,1
506,"Quick, Mrs. Frederick Charles (Jane Richards)",26360,33.0,26.0000,3,2,1
530,"Quick, Miss. Phyllis May",26360,2.0,26.0000,3,2,1
520,"Perreault, Miss. Anne",12749,30.0,93.5000,1,1,1
820,"Hays, Mrs. Charles Melville (Clara Jennings Gr...",12749,52.0,93.5000,3,1,1
